<a href="https://colab.research.google.com/github/solobala/STPY36/blob/main/STPY36_DZ1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции «Базовые понятия статистики»

##### Импортируем библиотеки для работы с данными

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np

##### Импортируем библиотеки для работы с описанием столбцов

In [ ]:
import requests as rq
from bs4 import BeautifulSoup

## Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения.

*  1 столбец - 'surgery?' - операция? Значения: 1 - лошадь была прооперирована; 2 - лечение проводилось без операции.
*  2 столбец: Age: 1 = Adult horse (взрослая лошадь); 2 = Young (< 6 months) (молодая лошадь)

*  3 столбец: 'rectal temperature' - температура.Нормальная температура составляет 37,8.
*   4 столбец - 'pulse' частота сердечных сокращений в ударах в минуту.30-40 -  для взрослых животных. У спортивных лошадей - 20-25, у больных >40

*   5 столбец - 'respiratory rate' - частота дыхания. норма составляет от 8 - 10 
*   6 столбец - 'temperature of extremities' - температура конечностей,   Значения: 1 = Нормальный; 2 = Теплый; 3 = Прохладный; 4 = Холодный

*   7 столбец - 'pain' - субъективное суждение об уровне боли лошади
 Значения: 1 = отсутствие боли (сигнал тревоги!), 2 = подавленный, 3 = периодич. легкая боль, 4 = периодич. сильная боль, 5 = непрерывная сильная боль. - should NOT be treated as a ordered or discrete variable!
*   8 столбец - 'Outcome' - исход # Значения: 1 = жив; 2 = умер; 3 = был подвергнут эвтаназии

##### Читаем описание столбцов, определяем, какие нам понадобятся, предполагаем, какие типы данных должны получиться и их возможные диапазоны

In [ ]:
req =rq.get('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names').text
soup = BeautifulSoup(req)
soup

<html><body><p>1. TItle: Horse Colic database

2. Source Information
   -- Creators: Mary McLeish &amp; Matt Cecile
	  	Department of Computer Science
		University of Guelph
		Guelph, Ontario, Canada N1G 2W1
		mdmcleish@water.waterloo.edu
   -- Donor:    Will Taylor (taylor@pluto.arc.nasa.gov)
   -- Date:     8/6/89

3. Past Usage:
   -- Unknown

4. Relevant Information:

   -- 2 data files
      -- horse-colic.data: 300 training instances
      -- horse-colic.test: 68 test instances
   -- Possible class attributes: 24 (whether lesion is surgical)
     -- others include: 23, 25, 26, and 27
   -- Many Data types: (continuous, discrete, and nominal)

5. Number of Instances: 368 (300 for training, 68 for testing)

6. Number of attributes: 28

7. Attribute Information:

  1:  surgery?
          1 = Yes, it had surgery
          2 = It was treated without surgery

  2:  Age
          1 = Adult horse
          2 = Young (&lt; 6 months)

  3:  Hospital Number
          - numeric id
          

##### Читаем файл с данными из 8 столбцов в датафрейм, даем столбцам имена,  обрабатываем '?' как NAN

In [ ]:
names =['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']
df = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', usecols =[0,1,3,4,5,6,10,22],names = names, dtype ={'surgery?': 'str', 'Age': 'str', 'temperature of extremities' : 'str', 'pain' : 'str', 'outcome': 'str'}, na_values ='?',sep = ',')
#df = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', usecols =[0,1,3,4,5,6,10,22],names = names,  na_values ='?',sep = ',')

In [ ]:
df.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2,1,38.5,66.0,28.0,3,5,2
1,1,1,39.2,88.0,20.0,NaN,3,3
2,2,1,38.3,40.0,24.0,1,3,1
3,1,9,39.1,164.0,84.0,4,2,2
4,2,1,37.3,104.0,35.0,NaN,NaN,2


## Задание 2. Первичное изучение данных

##### Анализируем значения по столбцам, делаем выводы о кол-ве пропусков

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    object 
 1   Age                         300 non-null    object 
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    object 
 6   pain                        245 non-null    object 
 7   outcome                     299 non-null    object 
dtypes: float64(3), object(5)
memory usage: 18.9+ KB


##### Делаем выводы о типе данных: 
  ##### **Качественные** - surgery?, Age, temperature of extremities, pain, outcome. Несмотря на то , что 'pain' выглядит как порядковая категориальная величина (есть градация боли от 1 до 5), в описании данных есть указание на то, что pain should NOT be treated as a ordered or discrete variable, т.к это субъективная оценка + возможно было обезболивание
  ##### **Количественные**, непрерывные - rectal temperature, pulse, respiratory rate.

###### Для каждого из  признаков определим к-во уникальных значений 

In [ ]:
unique_counts = pd.DataFrame.from_records([(col, df[col].nunique()) for col in df.columns],
                                          columns=['Наименование признака', 'Кол-во уникальных значений']).sort_values(by=['Кол-во уникальных значений'])
unique_counts                                          

,Наименование признака,Кол-во уникальных значений
0,surgery?,2
1,Age,2
7,outcome,3
5,temperature of extremities,4
6,pain,5
2,rectal temperature,40
4,respiratory rate,40
3,pulse,52


##### Рассчитываем базовые статистики по количественным признакам

In [ ]:
df.describe(exclude = ['O']) 

,rectal temperature,pulse,respiratory rate
count,240.000000,276.000000,242.000000
mean,38.167917,71.913043,30.417355
std,0.732289,28.630557,17.642231
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,18.500000
50%,38.200000,64.000000,24.500000
75%,38.500000,88.000000,36.000000
max,40.800000,184.000000,96.000000


##### В результатах describe() нет расчетов размаха, дисперсии, межквартильного размаха, поэтому написана  ф-я get_stat(names), которую применим к количественным признакам

In [ ]:
def get_stat(names):
  """
  Функция для расчета и печати основных статистик.
  на входе - список названий столбцов
  на выходе- к-во признаков серии за вычетом NAN и 10 основных статистик - 
  max, min, range, mean, mode, std, var, Q1, Q3, iqr
  """
  for name in names:
    count_ = df[name].count()
    max_ = df[name].max() 
    min_ = df[name].min()
    range_ = round(df[name].max() - df[name].min(),1)
    mean_ = df[name].mean()
    mediana_ = df[name].quantile(0.50)
    mode_ = df[name].mode()[0]
    std_ = df[name].std()
    var_ = df[name].var()
    Q1 = df[name].quantile(0.25)
    Q3 = df[name].quantile(0.75) 
    iqr = Q3-Q1
    print(f'{name}:')
    print(f'count = {count_}, max = {max_}, min = {min_}, range = {range_}, mean = {mean_}, mediana = {mediana_},  mode = {mode_}, std = {std_}, var = {var_}, Q1 = {Q1},  Q3 = {Q3}, IQR = {iqr}')
    print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------')


In [ ]:
get_stat(names[2:5])

rectal temperature:
count = 240, max = 40.8, min = 35.4, range = 5.4, mean = 38.16791666666667, mediana = 38.2,  mode = 38.0, std = 0.7322886641121578, var = 0.5362466875871686, Q1 = 37.8,  Q3 = 38.5, IQR = 0.7000000000000028
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
pulse:
count = 276, max = 184.0, min = 30.0, range = 154.0, mean = 71.91304347826087, mediana = 64.0,  mode = 48.0, std = 28.630556660735, var = 819.7087747035573, Q1 = 48.0,  Q3 = 88.0, IQR = 40.0
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
respiratory rate:
count = 242, max = 96.0, min = 8.0, range = 88.0, mean = 30.417355371900825, mediana = 24.5,  mode = 20.0, std = 17.64223138513467, var = 311.2483282466308, Q1 = 18.5,  Q3 = 36.0, IQR = 17.5
---------------------------------------

###### Для каждого из количественных признаков определяем распределение уникальных значений

In [ ]:
for name in names[2:5]:
  print(df[name].value_counts())
  print('-----------------------')
  

38.0    26
38.5    19
38.3    18
37.8    17
38.2    16
38.1    12
37.5    12
38.6    12
38.4    11
37.9     7
37.2     7
37.6     7
37.7     7
38.7     7
37.3     6
38.8     6
39.2     5
39.5     4
39.3     4
38.9     4
39.0     4
39.4     3
37.1     3
37.4     3
40.3     2
37.0     2
39.1     2
36.5     2
36.9     1
40.8     1
36.6     1
36.1     1
35.4     1
36.0     1
36.8     1
39.6     1
39.9     1
36.4     1
39.7     1
40.0     1
Name: rectal temperature, dtype: int64
-----------------------
48.0     28
60.0     25
40.0     18
52.0     12
88.0     12
44.0     11
100.0    11
72.0     11
120.0    10
42.0     10
64.0      8
96.0      8
84.0      8
80.0      8
54.0      7
56.0      7
50.0      6
66.0      6
104.0     5
68.0      5
92.0      4
70.0      4
76.0      4
108.0     3
86.0      3
78.0      3
90.0      3
36.0      3
112.0     3
114.0     2
45.0      2
140.0     2
124.0     2
150.0     2
130.0     2
30.0      2
132.0     1
129.0     1
184.0     1
75.0      1
65.0      1
164.0

##### Поиск выбросов по количественным признакам с заданием граничных значений вручную

In [ ]:
def get_stat_without_outliers(names, boundaries):
  """
  функция для расчета и сравнения среднего арифметического, медианы и моды по всем значениям списка names и без выбросов
  граничные значения для определения выбросов передаются в boundaries в виде списка кортежей
  """
  i = 0
  for name in names:
    low = boundaries[i][0]
    high = boundaries[i][1]
    print(f'{name}: ')
    print('===========================')
    mean_old = df[name].mean()
    mean_new = df[(df[name] > low) & (df[name] < high)][name].mean()
    mediana_old = df[name].median()
    mediana_new = df[(df[name] > low) & (df[name] < high)][name].median()
    moda_old = df[name].mode()[0]
    moda_new = df[(df[name] > low) & (df[name] < high)][name].mode()[0]
    print(f'среднее арифметическое: {mean_old}, без выбросов: {mean_new}')
    print(f'Медиана: {mediana_old}, без выбросов: {mediana_new}')
    print(f'Мода: {moda_old}, без выбросов: {moda_new}')
    print('----------------------------------------------------------------')
    i+=1

In [ ]:
boundaries = list()
boundaries.append((36.6, 38.5))
boundaries.append((30, 40))
boundaries.append((8, 10))
get_stat_without_outliers(names[2:5],boundaries)

rectal temperature: 
среднее арифметическое: 38.16791666666667, без выбросов: 37.87564102564102
Медиана: 38.2, без выбросов: 38.0
Мода: 38.0, без выбросов: 38.0
----------------------------------------------------------------
pulse: 
среднее арифметическое: 71.91304347826087, без выбросов: 36.5
Медиана: 64.0, без выбросов: 36.0
Мода: 48.0, без выбросов: 36.0
----------------------------------------------------------------
respiratory rate: 
среднее арифметическое: 30.417355371900825, без выбросов: 9.0
Медиана: 24.5, без выбросов: 9.0
Мода: 20.0, без выбросов: 9.0
----------------------------------------------------------------


##### Определение выбросов по межквартильному размаху

###### rectal temperature

In [ ]:
# rectal temperature -74 СтрокИ отнесены к выбросам
q1 = df['rectal temperature'].quantile(0.25)
q3 = df['rectal temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['rectal temperature'].between(lower_bound, upper_bound, 'both')]
remove_outliers # без выбросов

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2,1,38.5,66.0,28.0,3,5,2
1,1,1,39.2,88.0,20.0,NaN,3,3
2,2,1,38.3,40.0,24.0,1,3,1
3,1,9,39.1,164.0,84.0,4,2,2
4,2,1,37.3,104.0,35.0,NaN,NaN,2
...,...,...,...,...,...,...,...,...
292,1,1,37.0,66.0,20.0,1,4,2
294,2,1,38.5,40.0,16.0,1,2,1
296,2,1,37.2,72.0,24.0,3,4,3
297,1,1,37.5,72.0,30.0,4,4,2


In [ ]:
mean_remove_outliers = remove_outliers['rectal temperature'].mean()
mediana_remove_outliers = remove_outliers['rectal temperature'].median()
print(f'Среднее значение по rectal temperature по строкам без выбросов: {mean_remove_outliers}')
print(f'Медиана по rectal temperature по строкам без выбросов: {mediana_remove_outliers}')

Среднее значение по rectal temperature по строкам без выбросов: 38.16902654867257
Медиана по rectal temperature по строкам без выбросов: 38.2


In [ ]:
outliers = df[~df['rectal temperature'].between(lower_bound, upper_bound, 'both')] # а что это за выбросы?
outliers

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
5,2,1,NaN,NaN,NaN,2,2,1
7,1,1,NaN,60.0,NaN,3,NaN,2
8,2,1,NaN,80.0,36.0,3,4,3
16,1,9,NaN,128.0,36.0,3,4,2
20,1,1,39.9,72.0,60.0,1,5,1
...,...,...,...,...,...,...,...,...
282,1,1,NaN,70.0,16.0,3,2,2
288,1,1,NaN,NaN,NaN,NaN,NaN,1
293,1,1,NaN,78.0,24.0,3,NaN,3
295,1,1,NaN,120.0,70.0,4,2,3


In [ ]:
mean_outliers = outliers['rectal temperature'].mean()
mediana_outliers = outliers['rectal temperature'].median()
print(f'Среднее значение по rectal temperature для выбросов: {mean_outliers}')
print(f'Медиана по rectal temperature для выбросов: {mediana_outliers}')

Среднее значение по rectal temperature для выбросов: 38.14999999999999
Медиана по rectal temperature для выбросов: 38.1


Для признака rectal temperature Все выбросы отнесены либо к NAN, либо равномерно ниже-выше среднего значения rectal temperature.
И медиана, и среднее  в выборке без выбросов и с выбросами имеют близкие значения. при этом медиана в общей выборке и выборке без выбросов имеет одинаковое значение

###### pulse

In [ ]:
#  pulse - 29 строк отнесены к выбросам ( более 10% от общего к-ва)
q1 = df['pulse'].quantile(0.25)
q3 = df['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['pulse'].between(lower_bound, upper_bound, 'both')]
remove_outliers # без выбросов
#display(pd.DataFrame(remove_outliers)) можно и так


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2,1,38.5,66.0,28.0,3,5,2
1,1,1,39.2,88.0,20.0,NaN,3,3
2,2,1,38.3,40.0,24.0,1,3,1
4,2,1,37.3,104.0,35.0,NaN,NaN,2
6,1,1,37.9,48.0,16.0,1,3,1
...,...,...,...,...,...,...,...,...
295,1,1,NaN,120.0,70.0,4,2,3
296,2,1,37.2,72.0,24.0,3,4,3
297,1,1,37.5,72.0,30.0,4,4,2
298,1,1,36.5,100.0,24.0,3,3,1


In [ ]:
mean_remove_outliers = remove_outliers['pulse'].mean()
mediana_remove_outliers = remove_outliers['pulse'].median()
print(f'Среднее значение по pulse по строкам без выбросов: {mean_remove_outliers}')
print(f'Медиана по pulse по строкам без выбросов: {mediana_remove_outliers}')

Среднее значение по pulse по строкам без выбросов: 70.25830258302582
Медиана по pulse по строкам без выбросов: 64.0


In [ ]:
outliers = df[~df['pulse'].between(lower_bound, upper_bound, 'both')] # а что это за выбросы?
outliers

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1,9,39.1,164.0,84.0,4,2,2
5,2,1,NaN,NaN,NaN,2,2,1
28,1,1,NaN,NaN,NaN,NaN,NaN,2
41,2,9,39.0,150.0,72.0,NaN,NaN,1
52,2,1,NaN,NaN,NaN,1,1,1
55,1,9,38.6,160.0,20.0,3,3,2
56,1,1,NaN,NaN,NaN,NaN,NaN,1
58,1,1,NaN,NaN,20.0,4,5,2
74,1,9,NaN,NaN,NaN,NaN,NaN,2
78,1,1,NaN,NaN,NaN,3,5,2


In [ ]:
mean_outliers = outliers['pulse'].mean()
mediana_outliers = outliers['pulse'].median()
print(f'Среднее значение по pulse для выбросов: {mean_outliers}')
print(f'Медиана по pulse для выбросов: {mediana_outliers}')

Среднее значение по pulse для выбросов: 161.6
Медиана по pulse для выбросов: 160.0


###### Для признака pulse все выбросы отнесены либо к NAN, либо к верхней границе значений пульса. что объяснимо - у больных животных пульс повышен. Среднее и медиана в выборке без выбросов и в выбросах очень сильно различаются, при этом медиана в общей выборке и выборке без выбросов имеет одинаковое значение

###### respiratory rate

In [ ]:
# 75 Строк отнесены к выбросам
q1 = df['respiratory rate'].quantile(0.25)
q3 = df['respiratory rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['respiratory rate'].between(lower_bound, upper_bound, 'both')]
remove_outliers # без выбросов

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2,1,38.5,66.0,28.0,3,5,2
1,1,1,39.2,88.0,20.0,NaN,3,3
2,2,1,38.3,40.0,24.0,1,3,1
4,2,1,37.3,104.0,35.0,NaN,NaN,2
6,1,1,37.9,48.0,16.0,1,3,1
...,...,...,...,...,...,...,...,...
294,2,1,38.5,40.0,16.0,1,2,1
296,2,1,37.2,72.0,24.0,3,4,3
297,1,1,37.5,72.0,30.0,4,4,2
298,1,1,36.5,100.0,24.0,3,3,1


In [ ]:
mean_remove_outliers = remove_outliers['respiratory rate'].mean()
mediana_remove_outliers = remove_outliers['respiratory rate'].median()
print(f'Среднее значение по respiratory rate по строкам без выбросов: {mean_remove_outliers}')
print(f'Медиана по respiratory rate по строкам без выбросов: {mediana_remove_outliers}')

Среднее значение по respiratory rate по строкам без выбросов: 26.715555555555557
Медиана по respiratory rate по строкам без выбросов: 24.0


In [ ]:
outliners = df[~df['respiratory rate'].between(lower_bound, upper_bound, 'both')] # а что это за выбросы?
outliners

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1,9,39.1,164.0,84.0,4,2,2
5,2,1,NaN,NaN,NaN,2,2,1
7,1,1,NaN,60.0,NaN,3,NaN,2
9,2,9,38.3,90.0,NaN,1,5,1
28,1,1,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...
274,1,1,NaN,76.0,NaN,NaN,NaN,3
281,2,1,40.0,78.0,NaN,3,2,2
284,2,1,38.5,54.0,NaN,1,3,1
288,1,1,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
mean_outliers = outliers['respiratory rate'].mean()
mediana_outliers = outliers['respiratory rate'].median()
print(f'Среднее значение по respiratory rate для выбросов: {mean_outliers}')
print(f'Медиана по respiratory rate для выбросов: {mediana_outliers}')

Среднее значение по respiratory rate для выбросов: 41.18181818181818
Медиана по respiratory rate для выбросов: 24.0


###### Для признака respiratory rate Все выбросы отнесены либо к NAN, либо к верхней границе значений respiratory rate, что объяснимо - у больных лошадей respiratory rate повышен. Среднее и медиана в выборке без выбросов и в выбросах очень сильно различаются, при этом медиана в общей выборке, выборке без выбросов и в выбросах имеет одинаковое значение

## Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

##### Распределение пропусков данных по столбцам в % соотношении

In [ ]:
display(pd.DataFrame(df.isna().mean() * 100).round(2))

,0
surgery?,0.33
Age,0.00
rectal temperature,20.00
pulse,8.00
respiratory rate,19.33
temperature of extremities,18.67
pain,18.33
outcome,0.33


##### Количество пропусков данных по столбцам в абсолютном выражении

In [ ]:
display(pd.DataFrame(df.isna().sum()))

,0
surgery?,1
Age,0
rectal temperature,60
pulse,24
respiratory rate,58
temperature of extremities,56
pain,55
outcome,1


##### Наибольшее количество пропусков - в данных количественных признаков rectal temperature, respiratory rate и pulse, а также  У категориальных признаков temperature of extremities и pain.
##### У категориального признака Age - нет пропущенных данных.У  surgery? и outcome - по 1 пропуску. Посмотрим на эту строку

####1. Удаление строк с пропусками

In [ ]:
df[df['surgery?'].isnull()]

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
132,NaN,1,38.0,48.0,20.0,3,4,NaN


Очевидно, эта строка - запись о лошади, которая на момент записи все еще проходила лечение ( значение outcome также не определено). Логично удалить эту строку из датафрейма

In [ ]:
df.dropna(subset=['surgery?', 'outcome'], inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    object 
 1   Age                         299 non-null    object 
 2   rectal temperature          239 non-null    float64
 3   pulse                       275 non-null    float64
 4   respiratory rate            241 non-null    float64
 5   temperature of extremities  243 non-null    object 
 6   pain                        244 non-null    object 
 7   outcome                     299 non-null    object 
dtypes: float64(3), object(5)
memory usage: 21.0+ KB


###### Что будет, если удалить все строки, где имеется хотя бы один пропуск? 

In [ ]:
df.copy().dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    154 non-null    object 
 1   Age                         154 non-null    object 
 2   rectal temperature          154 non-null    float64
 3   pulse                       154 non-null    float64
 4   respiratory rate            154 non-null    float64
 5   temperature of extremities  154 non-null    object 
 6   pain                        154 non-null    object 
 7   outcome                     154 non-null    object 
dtypes: float64(3), object(5)
memory usage: 10.8+ KB


######Остается чуть больше половины строк - не используем этот метод

#### *Замечание по данными Age:*
###### Несмотря на то, что в описании данных есть указания о диапазоне значений Age, в выборке нет значения 2, присваиваемого молодым лошадям, зато есть 9. Для нас это не принципиально,т.к это категориальный не порядковый признак, и замену можно не делать, просто воспринимаем 9 как молодых лошадей Доля молодых лошадей - менее 10%

In [ ]:
df['Age'].value_counts()

1    275
9     24
Name: Age, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    object 
 1   Age                         299 non-null    object 
 2   rectal temperature          239 non-null    float64
 3   pulse                       275 non-null    float64
 4   respiratory rate            241 non-null    float64
 5   temperature of extremities  243 non-null    object 
 6   pain                        244 non-null    object 
 7   outcome                     299 non-null    object 
dtypes: float64(3), object(5)
memory usage: 21.0+ KB


###### Что, если удалить строки, в которых отсутсвуют значения 2 и более количественных признаков?

In [ ]:
df.copy().dropna(subset = names[2:5], thresh = 1) 


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2,1,38.5,66.0,28.0,3,5,2
1,1,1,39.2,88.0,20.0,NaN,3,3
2,2,1,38.3,40.0,24.0,1,3,1
3,1,9,39.1,164.0,84.0,4,2,2
4,2,1,37.3,104.0,35.0,NaN,NaN,2
...,...,...,...,...,...,...,...,...
295,1,1,NaN,120.0,70.0,4,2,3
296,2,1,37.2,72.0,24.0,3,4,3
297,1,1,37.5,72.0,30.0,4,4,2
298,1,1,36.5,100.0,24.0,3,3,1


###### Происходит удаление 4,5 % строк, что приемлемо, но попробуем вариант с заментой пропусков

## 2. Замена пропусков

### Сделаем замены пропусков NaN для количественных признаков  rectal temperature, respiratory rate и pulse.

*   ###### при определении выбросов  у rectal_temperature выбросы равномерно распределялись выше-ниже среднего значения, проэтому выполним замену медианой ,
*   ###### у respiratory rate и pulse выбросы попадали в зону верхних значений, поэтому  рассчитаем  средние арифметические, медианы и моды по этим признакам в разрезе категориальных признаков surgery? и  pain

##### Мы не можем исключать, что медианный пульс у прооперированных и непрооперированных лошадей  отличался

In [ ]:

print(df.groupby('surgery?')['pulse'].median())
fill_median_by_surgery = df.copy()
fill_median_by_surgery['pulse'] = fill_median_by_surgery['pulse'].fillna(df.groupby('surgery?')['pulse'].transform('median'))

surgery?
1    70.0
2    56.0
Name: pulse, dtype: float64


###### Версия подтвердилась

##### А может быть, пульс зависит и от возраста?

In [ ]:
print(df.groupby(['surgery?', 'Age'])['pulse'].median())
fill_median_by_groups = df.copy()
fill_median_by_groups['pulse'] = fill_median_by_groups['pulse'].fillna(df.groupby(['surgery?', 'Age'])['pulse'].transform('median'))

surgery?  Age
1         1       66.0
          9      129.0
2         1       54.0
          9       91.0
Name: pulse, dtype: float64


######Версия подтвердилась

##### Мы не можем исключать, что медианная  respiratory rate у молодых и взрослых лошадей  отличался

In [ ]:
print(df.groupby('Age')['respiratory rate'].median())
fill_median_by_surgery = df.copy()
fill_median_by_surgery['respiratory rate'] = fill_median_by_surgery['respiratory rate'].fillna(df.groupby('Age')['respiratory rate'].transform('median'))

Age
1    24.0
9    49.0
Name: respiratory rate, dtype: float64


######Версия подтвердилась

###### Возможно, respiratory rate зависит и от того, проводилась ли операция

In [ ]:
print(df.groupby(['Age','surgery?'])['respiratory rate'].median())
fill_median_by_groups = df.copy()
fill_median_by_groups['respiratory rate'] = fill_median_by_groups['respiratory rate'].fillna(df.groupby(['Age','surgery?'])['respiratory rate'].transform('median'))

Age  surgery?
1    1           24.0
     2           24.0
9    1           60.0
     2           30.0
Name: respiratory rate, dtype: float64


######Получается, что respiratory rate У молодых лошадей зависит  от оперирования, а у старых - нет

#####Итак, выполним следующие замены:
#####В столбце rectal temperature заменим NAN на медиану,
#####в столбце pulse - выполним замену на медианы, рассчитанные с группировкой по surgery? и Age,
#####в столбце respiratory rate также выполним замену - для молодых лошадей - на медианы с группировкой по Age, surgery?, для взрослых - на медиану с группировкой по Age

In [ ]:
# тренируемся
fill_median = df.copy()
fill_median['rectal temperature'] = fill_median['rectal temperature'].fillna(df['rectal temperature'].median())
print(fill_median['rectal temperature'].isna().sum())
print(fill_median['rectal temperature'].unique())

0
[38.5 39.2 38.3 39.1 37.3 38.2 37.9 38.1 37.2 38.  37.6 37.5 39.4 39.9
 38.4 38.6 37.8 37.7 39.  35.4 38.9 37.4 40.3 37.  39.7 38.7 36.4 38.8
 39.6 36.8 39.5 36.5 36.  37.1 39.3 36.1 36.6 40.8 36.9 40. ]


In [ ]:
# Заполняем пропуски в rectal temperature датафрейма
df['rectal temperature'] = df['rectal temperature'].fillna(df['rectal temperature'].median())
print(df['rectal temperature'].isna().sum())
print(df['rectal temperature'].unique())

0
[38.5 39.2 38.3 39.1 37.3 38.2 37.9 38.1 37.2 38.  37.6 37.5 39.4 39.9
 38.4 38.6 37.8 37.7 39.  35.4 38.9 37.4 40.3 37.  39.7 38.7 36.4 38.8
 39.6 36.8 39.5 36.5 36.  37.1 39.3 36.1 36.6 40.8 36.9 40. ]


In [ ]:
# тренируемся
fill_median_by_groups = df.copy()
fill_median_by_groups['pulse'] = fill_median_by_groups['pulse'].fillna(df.groupby(['surgery?', 'Age'])['pulse'].transform('median'))
print(fill_median_by_groups['pulse'].isna().sum())
print(fill_median_by_groups['pulse'].unique())

0
[ 66.  88.  40. 164. 104.  54.  48.  60.  80.  90.  72.  42.  92.  76.
  96. 128.  64. 110. 130. 108. 100. 112.  52. 146. 150. 120. 140.  84.
  46. 114. 160.  56. 129.  38.  98.  50.  44.  70.  78.  49.  30. 136.
 132.  65.  86.  68.  36.  45. 124. 184.  75.  82.]


In [ ]:
# Заполняем пропуски в pulse датафрейма
df['pulse'] = df['pulse'].fillna(df.groupby(['surgery?', 'Age'])['pulse'].transform('median'))
print(df['pulse'].isna().sum())
print(df['pulse'].unique())

0
[ 66.  88.  40. 164. 104.  54.  48.  60.  80.  90.  72.  42.  92.  76.
  96. 128.  64. 110. 130. 108. 100. 112.  52. 146. 150. 120. 140.  84.
  46. 114. 160.  56. 129.  38.  98.  50.  44.  70.  78.  49.  30. 136.
 132.  65.  86.  68.  36.  45. 124. 184.  75.  82.]


In [ ]:
# тренируемся
fill_median_by_groups = df.copy()
fill_median_by_groups['respiratory rate'] = fill_median_by_groups['respiratory rate'].fillna(df.groupby(['Age','surgery?'])['respiratory rate'].transform('median'))
print(fill_median_by_groups['respiratory rate'].isna().sum())
print(fill_median_by_groups['respiratory rate'].unique())

0
[28. 20. 24. 84. 35. 16. 36. 30. 12. 52. 48. 21. 60. 34. 42. 96. 72. 15.
 44. 32. 22. 18. 80. 40. 10. 51. 68. 66.  9. 14. 90.  8. 23. 58. 13. 70.
 26. 88. 25. 50.]


In [ ]:
# Заполняем пропуски в respiratory rate датафрейма
df['respiratory rate'] = df['respiratory rate'].fillna(df.groupby(['Age','surgery?'])['respiratory rate'].transform('median'))
print(df['respiratory rate'].isna().sum())
print(df['respiratory rate'].unique())

0
[28. 20. 24. 84. 35. 16. 36. 30. 12. 52. 48. 21. 60. 34. 42. 96. 72. 15.
 44. 32. 22. 18. 80. 40. 10. 51. 68. 66.  9. 14. 90.  8. 23. 58. 13. 70.
 26. 88. 25. 50.]


### Начнем заполнение пропусков в категориальных признаках. 
###Сделаем замены для категориальных признаков 'temperature of extremities' и 'pain' пропусков NaN на 'no_info'. 
###Не используем заполнение модой для данных признаков

In [ ]:
df['temperature of extremities'] = df['temperature of extremities'].fillna('no_info')
df['pain'] = df['pain'].fillna('no_info')

In [ ]:
print('temperature of extremities')
print('==================================')
print(df['temperature of extremities'].isna().sum())
print(df['temperature of extremities'].unique())
print(df['temperature of extremities'].value_counts())
print('---------------------------------')
print('pain')
print('==================================')
df['pain'] = df['pain'].fillna('no_info')
print(df['pain'].isna().sum())
print(df['pain'].unique())
print(df['temperature of extremities'].value_counts())

temperature of extremities
0
['3' 'no_info' '1' '4' '2']
3          108
1           78
no_info     56
2           30
4           27
Name: temperature of extremities, dtype: int64
---------------------------------
pain
0
['5' '3' '2' 'no_info' '4' '1']
3          108
1           78
no_info     56
2           30
4           27
Name: temperature of extremities, dtype: int64


###### Для признаков, которые определены как категориальные, а pandas восприняла как количественные, выполним замену типа на Categorical. Это позволит сократить используемый объем памяти и дополнительно получить доступ к методам категориальных признаков в последующих исследованиях. Здесь постановка задачи этого не требует, сделано для тренировки

In [ ]:
df['surgery?'] = pd.Categorical(df['surgery?'])
df['Age'] = pd.Categorical(df['Age'])
df['temperature of extremities'] = pd.Categorical(df['temperature of extremities'])
df['pain'] = pd.Categorical(df['pain'])
df['outcome'] = pd.Categorical(df['outcome'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   surgery?                    299 non-null    category
 1   Age                         299 non-null    category
 2   rectal temperature          299 non-null    float64 
 3   pulse                       299 non-null    float64 
 4   respiratory rate            299 non-null    float64 
 5   temperature of extremities  299 non-null    category
 6   pain                        299 non-null    category
 7   outcome                     299 non-null    category
dtypes: category(5), float64(3)
memory usage: 11.6 KB


In [ ]:
# Из-за ? в данных все стобцы кроме aGE воспринимаются Пандас как объекты (строки)
# можно при чтении в read_csv добавить опцию na_values ='?', а строковым столбцам явно задать тип данных
# Или уже в датафрейме выполнить преобразование типов с помощью to_numeric() или astype()
# df[['rectal temperature','respiratory rate', 'pulse']] = df[['rectal temperature','respiratory rate', 'pulse']].astype(float)
# df['rectal temperature'] = pd.to_numeric(df['rectal temperature'],errors='coerce')
# df['respiratory rate'] = pd.to_numeric(df['respiratory rate'],errors='coerce')
# df['pulse'] = pd.to_numeric(df['pulse'],errors='coerce')

